In [13]:
import pandas as pd
import numpy as np
from datetime import datetime

In [14]:
# Load your dataset
df = pd.read_csv('data/raw/combined_iasi_no2_meteo_2020_2025_local.csv')

# Check for exact duplicates across all columns
exact_duplicates = df.duplicated().sum()
print(f"Exact duplicates found: {exact_duplicates}")

# View duplicate rows
duplicate_rows = df[df.duplicated(keep=False)]
print(f"Total rows involved in duplication: {len(duplicate_rows)}")

# Check duplicates on specific key columns
key_duplicates = df.duplicated(subset=['datetime']).sum()
print(f"Duplicates based on time: {key_duplicates}")

Exact duplicates found: 0
Total rows involved in duplication: 0
Duplicates based on time: 0


In [15]:
# Remove exact duplicates (keep first occurrence)
df_cleaned = df.drop_duplicates(keep='first')

# For key-based duplicates, investigate first
duplicate_keys = df[df.duplicated(subset=['datetime'], keep=False)]
print("Duplicate key patterns:")
print(duplicate_keys.groupby(['datetime']).size().sort_values(ascending=False))

# Remove duplicates based on key columns after investigation
df_cleaned = df.drop_duplicates(subset=['datetime'], keep='first')

Duplicate key patterns:
Series([], dtype: int64)


In [16]:
with open('outputs/logs/data_cleaning_log.txt', 'w') as f:
    f.write(f"Duplicate removal on {pd.Timestamp.now()}\n")
    f.write(f"Original dataset: {len(df)} rows\n")
    f.write(f"Exact duplicates removed: {exact_duplicates}\n")
    f.write(f"Key-based duplicates removed: {key_duplicates}\n")
    f.write(f"Final dataset: {len(df_cleaned)} rows\n")

In [17]:
df.describe()

,location_id,sensors_id,lat,lon,value,temp_C,dewpoint_C,slp_hPa,wind_dir_deg,wind_speed_ms,precip_mm
count,22081.0,22081.0,22081.000000,22081.000000,22081.000000,48674.000000,48674.000000,48674.000000,48253.000000,6.0,48674.000000
mean,9369.0,28602.0,47.156800,27.574886,31.661637,119.342400,58.548198,10085.463286,31.269538,-99.0,165.729178
std,0.0,0.0,0.000035,0.000021,30.498318,97.136655,100.504995,1349.646573,101.462163,0.0,364.906361
min,9369.0,28602.0,47.156766,27.574866,-1.000000,-168.000000,-9999.000000,-9999.000000,1.000000,-99.0,0.000000
25%,9369.0,28602.0,47.156766,27.574866,15.896116,37.000000,0.000000,10122.000000,10.000000,-99.0,5.000000
50%,9369.0,28602.0,47.156766,27.574866,29.376155,116.000000,58.000000,10169.000000,26.000000,-99.0,8.000000
75%,9369.0,28602.0,47.156836,27.574908,43.917378,194.000000,124.000000,10226.000000,30.000000,-99.0,8.000000
max,9369.0,28602.0,47.156836,27.574908,2217.676463,391.000000,236.000000,10474.000000,999.000000,-99.0,999.000000


In [20]:
df.head

<bound method NDFrame.head of        location_id  sensors_id   location                   datetime  \
0              NaN         NaN        NaN  2020-01-01 02:00:00+02:00   
1              NaN         NaN        NaN  2020-01-01 03:00:00+02:00   
2              NaN         NaN        NaN  2020-01-01 04:00:00+02:00   
3              NaN         NaN        NaN  2020-01-01 05:00:00+02:00   
4              NaN         NaN        NaN  2020-01-01 06:00:00+02:00   
...            ...         ...        ...                        ...   
49213       9369.0     28602.0  IS-1-9369  2025-08-20 20:00:00+03:00   
49214       9369.0     28602.0  IS-1-9369  2025-08-20 21:00:00+03:00   
49215       9369.0     28602.0  IS-1-9369  2025-08-20 22:00:00+03:00   
49216       9369.0     28602.0  IS-1-9369  2025-08-20 23:00:00+03:00   
49217       9369.0     28602.0  IS-1-9369  2025-08-21 00:00:00+03:00   

             lat        lon parameter  units      value  temp_C  dewpoint_C  \
0            NaN        Na

| Column       | Description                                                                 | Example value              |
|--------------|-----------------------------------------------------------------------------|----------------------------|
| location_id  | Unique numeric identifier of the monitoring location                        | 9369                       |
| sensors_id   | Unique numeric identifier of the sensor within the location                 | 28602                      |
| location     | Station code (often country code + site code + location ID)                 | RO0083A-9369               |
| datetime     | Timestamp of measurement (ISO 8601 with timezone)                          | 2020-08-04T01:00:00+03:00  |
| lat          | Latitude coordinate of the monitoring location                             | 47.1567664986992           |
| lon          | Longitude coordinate of the monitoring location                            | 27.5748656243897           |
| parameter    | Pollutant measured (e.g., `no2`, `pm10`, `pm25`, `o3`, etc.)               | no2                        |
| units        | Units of measurement (varies by parameter)                                 | µg/m³                      |
| value        | Recorded measurement value of the pollutant                                | 51.44521273                |


| Column (raw)       | Description                                                                 | Units (raw)       | Missing code |
|--------------------|-----------------------------------------------------------------------------|------------------|--------------|
| year               | Year (4-digit)                                                             | YYYY             | –            |
| month              | Month (2-digit)                                                            | MM               | –            |
| day                | Day of month (2-digit)                                                     | DD               | –            |
| hour               | Hour of day (UTC, 0–23)                                                     | HH               | –            |
| air temperature    | Air temperature in **tenths of °C**                                         | 0.1 °C           | -9999        |
| dew point temp     | Dew point temperature in **tenths of °C**                                   | 0.1 °C           | -9999        |
| sea level pressure | Sea level pressure in **tenths of hPa**                                     | 0.1 hPa          | -9999        |
| wind direction     | Wind direction from true north (0–360)                                      | degrees          | -999         |
| wind speed         | Wind speed in **tenths of m/s**                                             | 0.1 m/s          | -9999        |
| sky cover          | Cloud cover indicator (coded, e.g., oktas or station code dependent)        | categorical/code | -9999        |
| precipitation      | Precipitation depth during the past hour                                    | mm               | -9999        |

In [ ]:
def identify_quality_issues(df):
    """Comprehensive data quality assessment"""
    issues = {}
    
    # Check for impossible values (domain-specific)
    if 'temp_C' in df.columns:
        impossible_temps = df[(df['temp_C'] < -500) | (df['temp_C'] > 700)]
        issues['impossible_temperatures'] = len(impossible_temps)
    
    if 'value' in df.columns:
        impossible_no2 = df[(df['value'] < 0 | (df['value'] > 300))]
        issues['impossible_no2'] = len(impossible_no2)
    
    # Check for future dates
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
        future_dates = df[df['timestamp'] > datetime.now()]
        issues['future_dates'] = len(future_dates)
    
    # Check for format inconsistencies
    for col in df.select_dtypes(include=['object']).columns:
        unique_patterns = df[col].astype(str).str.len().value_counts()
        if len(unique_patterns) > 10:  # Many different lengths suggest format issues
            issues[f'{col}_format_inconsistency'] = len(unique_patterns)
    
    return issues

# Run quality assessment
quality_report = identify_quality_issues(df_cleaned)
print("Data Quality Issues Found:")
for issue, count in quality_report.items():
    if count > 0:
        print(f"  {issue}: {count} records")

Data Quality Issues Found:
  impossible_no2: 3930 records


In [22]:
# Confirm that date time format is consistent
print(df_cleaned["datetime"].dtype)

# Coerce to datetime with timezone awareness
df_cleaned["datetime"] = pd.to_datetime(df_cleaned["datetime"], errors="coerce", utc=True)
# Convert from UTC to Iași local time (Europe/Bucharest)
df_cleaned["datetime"] = df_cleaned["datetime"].dt.tz_convert("Europe/Bucharest")

print(df_cleaned["datetime"].dtype)      # should show: datetime64[ns, Europe/Bucharest]
print(df_cleaned["datetime"].head(3))    # should print like: 2025-08-20 20:00:00+03:00

object
datetime64[ns, Europe/Bucharest]
0   2020-01-01 02:00:00+02:00
1   2020-01-01 03:00:00+02:00
2   2020-01-01 04:00:00+02:00
Name: datetime, dtype: datetime64[ns, Europe/Bucharest]


In [23]:
with open('outputs/logs/data_cleaning_log.txt', 'w') as f:
    f.write(f"Timestamp format standardization on {pd.Timestamp.now()}\n")
    f.write(f"Original datetime type: {df["datetime"].dtype}\n")
    f.write(f"Final type: {df_cleaned["datetime"].dtype}\n")